<a href="https://colab.research.google.com/github/harvard-visionlab/neuro_science_fiction/blob/main/2022/mitchell_feature_modeling_class_dropFeatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NeuroScienceFiction Mind Reading Data Analysis

In [ ]:
!rm -R NeuroSciFi-Workshop
!rm -R brain_data

# setup (run this section first)

## download data & precomputed results

In [ ]:
%%capture
!wget -c https://www.dropbox.com/s/74s566ppu1nbo74/brain_data.tar.gz
!tar -xvf brain_data.tar.gz
!rm brain_data.tar.gz

In [ ]:
%%capture
!mkdir -p /content/NeuroSciFi-Workshop/
!rm -R /content/NeuroSciFi-Workshop/
!wget -c https://www.dropbox.com/s/pox900f93y6pkin/FeatureRatings.tar.gz
!mkdir -p /content/NeuroSciFi-Workshop/
!tar -xvf FeatureRatings.tar.gz
!mv FeatureRatings ./NeuroSciFi-Workshop/FeatureRatings
!rm -R FeatureRatings.tar.gz

In [ ]:
%%capture
!wget -c https://www.dropbox.com/s/r4xm9qruw4dqb0f/Results-BrainPrediction.tar.gz
!tar -xvf Results-BrainPrediction.tar.gz
!mv Results-BrainPrediction ./NeuroSciFi-Workshop/Results-BrainPrediction
!rm -R Results-BrainPrediction.tar.gz

In [ ]:
%%capture
!wget -c https://www.dropbox.com/s/tyoagsr0g8nlurr/Results-ByFeature.tar.gz
!tar -xvf Results-ByFeature.tar.gz
!mkdir -p ./NeuroSciFi-Workshop/Results-ByFeature 
!rm -R ./NeuroSciFi-Workshop/Results-ByFeature
!mv Results-ByFeature ./NeuroSciFi-Workshop/Results-ByFeature
!rm -R Results-ByFeature.tar.gz

## analysis code

In [ ]:
%config InlineBackend.figure_format = 'retina'

In [ ]:
import os 

drive = '/content/NeuroSciFi-Workshop'
if not os.path.exists(drive):
  os.makedirs(drive)


### handle feature data

In [ ]:
import os
import pandas as pd 
from datetime import date
import numpy as np
import pandas as pd 
from collections import defaultdict
import seaborn as sns 
from pdb import set_trace 

def download_ratings(team_name, dropRaters=[], dropFeatures=[]):
  print(f"==> Downloading data: {team_name}")
  team_name = team_name.lower()
  folder = os.path.join(drive, 'FeatureRatings')
  if not os.path.exists(folder):
    os.makedirs(folder)
  filename = os.path.join(folder, f"{team_name}_Ratings.csv")
  url = f"https://scorsese.wjh.harvard.edu/turk/experiments/nsf/survey/{team_name}/data"
  df = pd.read_csv(url)
  df = df[~df.workerId.isin(dropRaters)]
  df = df[~df.featureName.isin(dropFeatures)]

  # drop rows for raters with incomplete datasets
  # assuming max_count of ratings is expected/desired count
  counts = df.groupby('workerId').rating.count()
  max_count = counts.max()
  dropRaters = counts[counts<max_count].index.values
  if len(dropRaters) > 0:
    print(f"==> Dropping incomplete datasets: {dropRaters}")
    df = df[~df.workerId.isin(dropRaters)]
  
  df.to_csv(filename, index=False)

  return df 

def load_ratings(team_name, dropFeatures=[]):
  team_name = team_name.lower()
  filename = os.path.join(drive, 'FeatureRatings', f"{team_name}_Ratings.csv")    
  df = pd.read_csv(filename)
  group_name = df.iloc[0].groupName
  num_items = len(df.itemName.unique())
  num_features = len(df.featureName.unique())
  num_raters = len(df.workerId.unique())

  print("="*50)
  print(f"FEATURE RATINGS: {group_name}")
  print(f"{num_items} items, {num_features} features, {num_raters} raters")
  print(date.today().strftime("%B %d, %Y"))
  print("="*50)

  items = sorted(df.itemName.unique())
  features = sorted(df.featureName.unique())
  raters = sorted(df.workerId.unique())
  num_rows = len(df)
  print("items:", items)
  print("features:", features)
  print("raters:", raters)
  expected_rows = len(items) * len(features) * len(raters)
  assert expected_rows == len(df), f"Oops, expected {expected_rows}, got {len(df)}"
  print("number of rows:", num_rows)

  return df

def get_feature_data(team_name, dropFeatures=[]):

  df = load_ratings(team_name, dropFeatures=dropFeatures)

  itemNames = sorted(df.itemName.unique())
  featureNames = sorted(df.featureName.unique())
  raters = sorted(df.workerId.unique())
  numItems = len(itemNames)
  numFeatures = len(featureNames)
  numRaters = len(raters)

  R = np.zeros((numItems,numFeatures))

  # itemNames in same order as brain data (groupbed by category)
  itemNames = ['bear', 'cat', 'cow', 'dog', 'horse', 'arm', 'eye', 'foot', 'hand',
              'leg', 'apartment', 'barn', 'church', 'house', 'igloo', 'arch',
              'chimney', 'closet', 'door', 'window', 'coat', 'dress', 'pants',
              'shirt', 'skirt', 'bed', 'chair', 'desk', 'dresser', 'table',
              'ant', 'bee', 'beetle', 'butterfly', 'fly', 'bottle', 'cup',
              'glass', 'knife', 'spoon', 'bell', 'key', 'refrigerator',
              'telephone', 'watch', 'chisel', 'hammer', 'pliers', 'saw',
              'screwdriver', 'carrot', 'celery', 'corn', 'lettuce', 'tomato',
              'airplane', 'bicycle', 'car', 'train', 'truck']

  for itemNum,itemName in enumerate(itemNames):
    for featureNum,featureName in enumerate(featureNames):
      subset = df[(df.itemName==itemName) & (df.featureName==featureName)]
      numRatings = len(subset)
      assert numRatings==numRaters, f"Oops, expected {numRaters} ratings, got {numRatings}"
      R[itemNum,featureNum] = subset.ratingScaled.mean()

  feature_data = dict(
      R=R,
      itemNames=np.asarray(itemNames, dtype='object'),
      featureNames=np.asarray(featureNames, dtype='object')
  )

  return feature_data

### brain-prediction and mind-reading

In [ ]:
from fastprogress import master_bar, progress_bar 
from scipy.spatial.distance import pdist, squareform
import numpy as np 
import scipy.io as sio
from scipy.stats import zscore
from types import SimpleNamespace 
from collections import defaultdict 
from scipy import stats 
from pdb import set_trace 

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, RidgeCV, LinearRegression
from sklearn.metrics import r2_score

In [ ]:
def pearson_dist(a,b):
  return 1 - stats.pearsonr(a,b)[0]

dissimilarity_functions = dict(
    pearson_dist=pearson_dist,
)

def compare_actual_predicted(a,p,dissimilarity_fun,accuracy_measure='combo'):
  dist11=dissimilarity_fun(p[0,:],a[0,:])
  dist22=dissimilarity_fun(p[1,:],a[1,:])
  dist12=dissimilarity_fun(p[0,:],a[1,:])
  dist21=dissimilarity_fun(p[1,:],a[0,:])

  if accuracy_measure == 'combo':
    # sum the dissimilarity scores for the 2 right and 2 wrong comparisons
    totalDistCorrectCombo = dist11+dist22;
    totalDistWrongCombo = dist12+dist21;
    
    # got it right if the dissimilarity score for the "right combo" is lower
    correct=float(totalDistCorrectCombo<totalDistWrongCombo)
    
  else:
    correct=(float(dist11<dist12) + float(dist22<dist21))/2 # percent correct, individual predictions

  return dict(
      dist11=dist11,
      dist22=dist22,
      dist12=dist12,
      dist21=dist21,
      correct=correct
  )

In [ ]:
def botastic_predict_features(prototype_features, prototype_scores, test_scores):
  '''
    Predict test_features based on similarity between test_scores
    and prototype_scores, given the prototype_features. Essentially, 
    the test_features are estimated to be a combination of the
    protype_features weighted by similarity between test_item and 
    each prototype.

    Based on bo's eye-movement code, where the protype_features are screen
    coordinates, the prototype_scores are eye-tracker calibration measurements,
    and the test scores are the current eye-position measurements. By measuring
    the similarity between test_scores and prototype_scores, you can
    infer the test_features (current eye-position in screen coordinates).

    But this can be generalized to our mind-reading task, where the 
    protype_features are features of objects, the prototype_scores are
    the neural resposnes to those objects, and the test_scores are 
    the neural responses to test objects. By measuring the similarity
    between test_brain_responses and prototype_brain_responses, you can
    infer the test_item_features.

    We can even flip this, and treat the neural responses as "features",
    and use similarity in feature-space

  '''
  num_proto = prototype_scores.shape[0]

  scores = np.concatenate([prototype_scores, test_scores])

  # pairwise distance between all scores
  dists = squareform(pdist(scores, metric='euclidean'))

  # all scores vs. prototypes only (a)
  d = dists[:,0:num_proto]

  # use least squares to extrapolate from the prototypes
  # x is the matrix that solves prototype_features = d_prototype @ x
  # e.g., feature[0,0] = sum( d[0] * x[:,0])
  # Item0's Feature0 is the similarity of Item0 to each prototype (d[0]), times some weight (x[:,0])
  # Item1's Feature0 is the similarity of Item1 to each prototype (d[1]), times those same weights (x[:,0])
  # ... etc
  # Here we use lstsq to find weights the work across all protypes, then apply to test items too
  x,resid,rank,s = np.linalg.lstsq(d[0:num_proto,:], prototype_features, rcond=None)

  # matrix multiple the full distance matrix (d) times the least squares solution
  tmp = (d @ x)

  # should recover the protype features
  extrap_prototype_features = tmp[0:num_proto,:]

  # finally the extrapolated test item features
  predicted_features = tmp[num_proto:,:]

  return predicted_features
  

In [ ]:
def prepare_brain_data(brain_data, num_voxels=None, zscore=False):
  D = brain_data['D']
  D = D.mean(axis=2) # average across repetitions/runs 
  if zscore:  
    D = zscore(D, axis=0, ddof=1) # z-score across items
  N = min(num_voxels,D.shape[1]) if num_voxels is not None else D.shape[1]

  # sortIdx comes from MATLAB, and is 1-indexed, so subtract 1
  sortIdx = brain_data['sortIdx'][0:N] - 1
  D = D[:,sortIdx] # % take the N most reliable voxels (consistent patterns across runs)

  return D 

def prepare_ratings(feature_data, shuffle=False):
  if feature_data['R'].shape[1]==60:
    R = np.transpose(feature_data['R'])
  else:
    R = feature_data['R']  

  if shuffle:
    R = np.array([r[np.random.permutation(r.shape[0])] for r in R])
  
  return R


In [ ]:
def fit_feature_model(numItems, item1, item2, D, R):
  # get train and test items
  all_items = np.array(range(numItems))
  test_items = ((all_items==item1) | (all_items==item2))
  train_items = test_items==False 

  # get the training item brain response and features
  trainBrainData=D[train_items,:]
  trainItemFeatures=R[train_items,:]
  scalerX = StandardScaler()
  scalerY = StandardScaler()
  trainX = scalerX.fit_transform(trainItemFeatures)
  trainY = scalerY.fit_transform(trainBrainData)

  # get the test item brain response and features 
  testBrainData=D[test_items,:]
  testItemFeatures=R[test_items,:]
  testX = scalerX.transform(testItemFeatures)
  testY = scalerY.transform(testBrainData)
  
  # learn feature->voxel mapping from training data 
  reg = LinearRegression(fit_intercept=False).fit(trainX, trainY)
  score = reg.score(trainX, trainY)  

  return reg, score, trainX, trainY, testX, testY

In [ ]:
def doBrainPredictionEncodingModel(reg, testX, testY, dissimilarity_fun, append_results):
  predBrainData = reg.predict(testX)
  predBrain2 = (testX @ np.transpose(reg.coef_)) + reg.intercept_
  assert np.allclose(predBrain2, predBrainData), "oops, no matchy"    

  task = 'brain_prediction'
  method = 'encoding_model'
  for scoring_method in ['individual', 'combo']:
    res = compare_actual_predicted(testY, 
                                   predBrainData,
                                   dissimilarity_fun,
                                   accuracy_measure=scoring_method)
    append_results(res, task, method, scoring_method)

def doMindReadingEncodingModel(reg, testX, testY, dissimilarity_fun, append_results):
  predFeatures = (testY @ reg.coef_)
  task = 'mind_reading'
  method = 'encoding_model'
  for scoring_method in ['individual', 'combo']:
    res = compare_actual_predicted(testX,
                                   predFeatures,
                                   dissimilarity_fun,
                                   accuracy_measure=scoring_method)
    append_results(res, task, method, scoring_method)

In [ ]:
def doBrainPredictionBotasticTemplates(trainX, trainY, testX, testY, 
                                       dissimilarity_fun, append_results):
  '''
    trainX: features of training items
    trainY: neural responses to training items 
    testX: features of test items
    testY: neural responses to test items
  '''
  predBrainData = botastic_predict_features(trainY, trainX, testX)
  task = 'brain_prediction'
  method = 'botastic_templates'
  for scoring_method in ['individual', 'combo']:
    res = compare_actual_predicted(testY,
                                   predBrainData,
                                   dissimilarity_fun,
                                   accuracy_measure=scoring_method)
    append_results(res, task, method, scoring_method)

def doMindReadingBotasticTemplates(trainX, trainY, testX, testY, 
                                   dissimilarity_fun, append_results):
  '''
    trainX: features of training items
    trainY: neural responses to training items 
    testX: features of test items
    testY: neural responses to test items
  '''
  predFeatures = botastic_predict_features(trainX, trainY, testY)
  task = 'mind_reading'
  method = 'botastic_templates'
  for scoring_method in ['individual', 'combo']:
    res = compare_actual_predicted(testX,
                                   predFeatures,
                                   dissimilarity_fun,
                                   accuracy_measure=scoring_method)
    append_results(res, task, method, scoring_method)    

In [ ]:
def doBrainAndFeaturePrediction(brain_data, feature_data, dissimilarity_fun,
                                prefs, shuffle_features=False, testIndividualFeatures=False, 
                                mb=None):
  '''Hold-two-out prediction of brain_data from feature_data (and vice-versa)

    featureData: [numItems,numFeatures] array representing the features of each item
    brainData: [numItems,numVoxels] array representing how much each voxel responds to each item

    For all possible pairs of items (60 items = 1770 pairs!), we hold out 
    the test pair, and use the remaining 58 items to learn the mapping
    between features and voxel responses (i.e., determine the beta weight
    for each feature and each voxel).

    We then use the learned mapping to predict responses for the two items.
    For each of those items, we multiply their feature values times the
    beta values for each voxel to determine how much each voxel will respond
    to each item. We then compare the predicted patterns to the actual patterns.
    
  '''

  # make sure items are in the same order for brain data and feature data,
  # otherwise this is a lost cause:
  assert all(brain_data['itemName'] == feature_data['itemNames']), "oops, names don't match!"

  # get some metadata from datasets
  categoryNum = brain_data['categoryNum']
  categoryName = brain_data['categoryName']
  itemName = brain_data['itemName']
  brain_sub = brain_data['brain_sub']
  featureNames = feature_data['featureNames']
  print("ANALYZING SUBJECT NUMBER:", brain_sub)

  # prepare brain activations (numConditions x numVoxels x numRepetitions)
  D = prepare_brain_data(brain_data, num_voxels=prefs.num_voxels, zscore=prefs.zscore_braindata)

  # prepare feature ratings
  R = prepare_ratings(feature_data, shuffle=shuffle_features)

  # get dissimilarity function
  dissimilarity_fun = dissimilarity_functions[prefs.sim_metric]

  # prepare results
  results = defaultdict(list) # track results 
  results_by_feature = defaultdict(list) # track how well each feature predicts
  all_betas=[]; # so we can visualize the "weights"

  # for all possible pairs of items
  numItems = D.shape[0]
  pb = progress_bar(range(1770), parent=mb) # 60 choose 2
  c = 0
  for item1 in range(0,numItems-1):
    for item2 in range(item1+1,numItems):
      if (item1==item2): continue
      pb.update(c)
      c += 1
      
      # fit the encoding model
      reg,score,trainX,trainY,testX,testY = fit_feature_model(numItems, item1, item2, D, R)

      # store the betas
      all_betas.append(reg.coef_)

      # wrapper to add rows to the results for different analysis methods
      def append_results(res, task, method, scoring_method):
        same_category = int(categoryNum[item1]==categoryNum[item2])

        results['team_name'].append(prefs.teamName)
        results['brain_subject'].append(brain_sub)
        results['item1_idx'].append(item1)
        results['item2_idx'].append(item2)
        results['item1_name'].append(itemName[item1])
        results['item2_name'].append(itemName[item2])
        results['item1_cat'].append(categoryName[item1])
        results['item2_cat'].append(categoryName[item2])
        results['itemPair'].append((item1,item2))
        results['same_category'].append(same_category)
        results['r2_score'].append(score.mean())

        results['task'].append(task)
        results['method'].append(method)
        results['scoring'].append(scoring_method)
        results['dist11'].append(res['dist11'])
        results['dist22'].append(res['dist22'])
        results['dist12'].append(res['dist12'])
        results['dist21'].append(res['dist21'])
        results['correct'].append(res['correct'])

      # ================================================
      #  Brain Prediction / Mind Reading
      # ================================================

      # predict brain response from features using feature-voxel encoding model
      doBrainPredictionEncodingModel(reg, testX, testY, dissimilarity_fun, append_results)

      # predict features (items) from brain response using feature-voxel encoding model
      doMindReadingEncodingModel(reg, testX, testY, dissimilarity_fun, append_results)
      
      # predict brain response using feature-similarity-weighted-neural-templates
      doBrainPredictionBotasticTemplates(trainX, trainY, testX, testY, dissimilarity_fun, append_results)

      # predict features (items) from neural-similarity-weighted-feature-templates
      doMindReadingBotasticTemplates(trainX, trainY, testX, testY, dissimilarity_fun, append_results)

      # ================================================
      #  analyze each feature independently
      # ================================================
      if testIndividualFeatures==True:
        for feat_num, feat_name in enumerate(progress_bar(featureNames, parent=mb)):
          reg = LinearRegression().fit(trainX[:,[feat_num]], trainY)
          r2_score = reg.score(trainX[:,[feat_num]], trainY)

          def append_results(res, task, method, scoring_method):
            same_category = int(categoryNum[item1]==categoryNum[item2])

            results_by_feature['team_name'].append(prefs.teamName)
            results_by_feature['feat_num'].append(feat_num)
            results_by_feature['feat_name'].append(feat_name)
            results_by_feature['brain_subject'].append(brain_sub)
            results_by_feature['item1_idx'].append(item1)
            results_by_feature['item2_idx'].append(item2)
            results_by_feature['item1_name'].append(itemName[item1])
            results_by_feature['item2_name'].append(itemName[item2])
            results_by_feature['item1_cat'].append(categoryName[item1])
            results_by_feature['item2_cat'].append(categoryName[item2])
            results_by_feature['itemPair'].append((item1,item2))
            results_by_feature['same_category'].append(same_category)
            results_by_feature['r2_score'].append(r2_score.mean())

            results_by_feature['task'].append(task)
            results_by_feature['method'].append(method)
            results_by_feature['scoring'].append(scoring_method)
            results_by_feature['dist11'].append(res['dist11'])
            results_by_feature['dist22'].append(res['dist22'])
            results_by_feature['dist12'].append(res['dist12'])
            results_by_feature['dist21'].append(res['dist21'])
            results_by_feature['correct'].append(res['correct'])
                    
          doBrainPredictionEncodingModel(reg, testX[:,[feat_num]], testY, dissimilarity_fun, append_results)
            
  return results, results_by_feature, all_betas

In [ ]:
import os
from types import SimpleNamespace

def loadmat(filename):
  D = sio.loadmat(filename, 
                  squeeze_me=True,
                  struct_as_record=True, 
                  simplify_cells=True)
  return D
  
def run_full_analysis(teamName, 
                      sub_list=[1,2,3,4,5,6,7,8,9], 
                      num_voxels=500, 
                      sim_metric='pearson_dist', 
                      zscore_braindata=False, 
                      testIndividualFeatures=True,
                      num_shuffles=100):
  '''Run the brain-prediction and mind-reading analyses, using both
  standard feature modeling and the botastic protype method. Include
  a shuffle test for good measure.'''
  prefs = SimpleNamespace(
      teamName = teamName,
      num_voxels = num_voxels,
      sim_metric=sim_metric, 
      zscore_braindata=zscore_braindata,
      num_shuffles=num_shuffles,
  )
  print(prefs)

  # load feature_data for this team
  # featureFileName = feature_sets[prefs.teamName]
  featureFileName = os.path.join(drive, 'FeatureData', f'{teamName}_Features.mat')
  feature_data = loadmat(featureFileName)
  dissimilarity_fun = dissimilarity_functions[prefs.sim_metric]

  # loop through brain subjects
  df = None
  feat_df = None
  shuffle_df = None
  mb = master_bar(sub_list)
  betas_by_subject = []
  brain_meta = []
  for brain_sub in mb:
    # load brain data for this subject
    brain_data = loadmat(f'./brain_data/data-science-P{brain_sub}_converted.mat')
    brain_data['brain_sub'] = brain_sub

    # do full brain-prediction / mind-reading analysis
    output = doBrainAndFeaturePrediction(brain_data, feature_data, dissimilarity_fun,
                                         prefs, testIndividualFeatures=testIndividualFeatures, 
                                         shuffle_features=False, mb=mb)
    
    # format and store the results
    results, results_by_feature, all_betas = output
    betas = np.stack(all_betas).mean(axis=0)
    betas_by_subject.append(betas)

    df_ = pd.DataFrame(results)
    df = pd.concat([df, df_])

    feat_df_ = pd.DataFrame(results_by_feature)
    feat_df = pd.concat([feat_df, feat_df_])

    meta = brain_data['meta']
    meta['sortIdx'] = brain_data['sortIdx']
    meta['voxelReliability'] = brain_data['voxelReliability']
    brain_meta.append(meta)

    # do a shuffle test
    # for shuffle_num in range(num_shuffles):
    #   output = doBrainAndFeaturePrediction(brain_data, feature_data, dissimilarity_fun,
    #                                        prefs, testIndividualFeatures=False, 
    #                                        shuffle_features=True, mb=mb)

  return dict(
      prefs=prefs, 
      df=df, 
      feat_df=feat_df, 
      betas=betas_by_subject,
      brain_meta=brain_meta,
      feature_data=feature_data
  )

def compute_prediction_summary(results, task, method, scoring):

  summary = defaultdict(list)
  sub_nums = sorted(results.brain_subject.unique())
  
  for sub_num in sub_nums:
    subset = results[(results.brain_subject==sub_num) & (results.task==task) & 
                     (results.method==method) & (results.scoring==scoring)]
    assert len(subset)==1770, f"Oops, expected 1770 rows, got {len(subset)}"
    summary['accuracyOverall'].append(subset.correct.mean())
    summary['accuracySameCat'].append(subset[subset.same_category==1].correct.mean())
    summary['accuracyDiffCat'].append(subset[subset.same_category==0].correct.mean())
    # summary['aveAccByFeature'].append()

  summary['accuracyOverall'] = np.array(summary['accuracyOverall'])
  summary['accuracySameCat'] = np.array(summary['accuracySameCat'])
  summary['accuracyDiffCat'] = np.array(summary['accuracyDiffCat'])

  summary['ave_accuracyOverall'] = summary['accuracyOverall'].mean()
  summary['ave_accuracySameCat'] = summary['accuracySameCat'].mean()
  summary['ave_accuracyDiffCat'] = summary['accuracyDiffCat'].mean()
  
  return summary

def save_full_results(results, folder=os.path.join(drive, 'Results-BrainPrediction')):
  
  if not os.path.exists(folder):
      os.makedirs(folder)

  prefs = results['prefs']
  df = results['df']
  feat_df = results['feat_df']
  betas = results['betas']
  
  if feat_df is not None:
    filename = f'{prefs.teamName}_{prefs.num_voxels}_results_features.pth.tar'
  else:
    filename = f'{prefs.teamName}_{prefs.num_voxels}_results.pth.tar'
  filename = os.path.join(folder, filename)

  torch.save(results, filename)

### comparison and visualization

In [ ]:
from glob import glob 
from scipy.stats import ttest_rel
import seaborn as sns 
import matplotlib.pyplot as plt 

def compare_to_mitchell(teamName, 
                        num_voxels=500, 
                        task="brain_prediction",
                        method="encoding_model",
                        score="individual",
                        accuracy='accuracyOverall',
                        title="",
                        ax=None, 
                        ylabel="percent correct"):

  sns.set(rc={
      'figure.figsize':(7,8),
      'axes.grid': False,
  })
  sns.set(font_scale = 1.5)
  sns.set_style("white")

  files = glob(os.path.join(drive, f'./Results-BrainPrediction/*{teamName}*{num_voxels}*features*'))
  assert len(files)==1, f"Oops, expected 1 file, got {len(files)}"
  results1 = torch.load(files[0])
  summary1 = compute_prediction_summary(results1['df'], task, method, score)
  

  files = glob(os.path.join(drive, f'./Results-BrainPrediction/*Mitchell*{num_voxels}*features*'))
  assert len(files)==1, f"Oops, expected 1 file, got {len(files)}"
  results2 = torch.load(files[0])
  summary2 = compute_prediction_summary(results2['df'], task, method, score)

  a = summary1[accuracy]
  b = summary2[accuracy]
  tval, pval = ttest_rel(a, b, axis=0, nan_policy='propagate', alternative='two-sided')
  df = len(a)-1
  # print(f"t({df})={tval:4.2f}, p={pval:4.3f}")
  p = "(n.s.)" if pval >= .05 else f"(p={pval:4.3f})"

  results = defaultdict(list)
  for brainSubject,(d1,d2) in enumerate(zip(a,b)):
    results['brainSubject'].append(brainSubject)
    results['teamName'].append(teamName)
    results['percent correct'].append(d1)
    
    results['brainSubject'].append(brainSubject)
    results['teamName'].append('Mitchell')
    results['percent correct'].append(d2)
  results = pd.DataFrame(results)

  mitchell = results[results.teamName=="Mitchell"]['percent correct'].mean()
  other = results[results.teamName!="Mitchell"]['percent correct'].mean()
  if other > mitchell:
    title = title + f"Congratulations {teamName}\nYou Defeated Mitchell! {p}"
  else:
    title = title + f"Sorry {teamName}\nMitchell Wins {p}"

  ax = sns.barplot(data=results, x="teamName", y="percent correct", ax=ax);
  if score=="individual":
    ax.set_ylim([.5,0.85]);
  else:
    ax.set_ylim([.5,0.95]);

  ax.set_ylabel(ylabel, fontsize=24);
  ax.set_xlabel("team name", fontsize=24);
  ax.set_title(title, fontsize=22);
  ax.yaxis.labelpad = 20
  ax.xaxis.labelpad = 20
  ax.title.set_position([.5, 1.02])

  return ax;

In [ ]:
from collections import defaultdict
from fastprogress import progress_bar
from functools import lru_cache
import matplotlib.pyplot as plt 

@lru_cache()
def get_voxel_data(folder='./brain_data', brain_subjects=[1,2,3,4,5,6,7,8,9]):
  data = dict()
  for brain_sub in progress_bar(brain_subjects):
    brain_data = sio.loadmat(os.path.join(folder, f'data-science-P{brain_sub}_converted.mat'),
                             squeeze_me=True,
                             struct_as_record=True, 
                             simplify_cells=True)
    data[brain_sub] = dict(
        meta=brain_data['meta'],
        sortIdx=brain_data['sortIdx'],
        voxelReliability=brain_data['voxelReliability']
    )
  return data

def get_gray_matter_cube(brain_data, brain_subjects=[1,2,3,4,5,6,7,8,9]):
  brainCube = []
  for brain_sub in brain_subjects:
    cube = np.zeros((51, 61, 23))
    meta = brain_data[brain_sub]['meta']
    usedVoxels = meta['coordToCol']
    cube = np.zeros(usedVoxels.shape)
    cube[usedVoxels>0] = .15
    brainCube.append(cube)
  brainCube = np.stack(brainCube, axis=3)
  return brainCube

@lru_cache()
def get_reliable_cube(brain_data, num_voxels, 
                      brain_subjects=[1,2,3,4,5,6,7,8,9]):
  dataCube = []
  
  for brain_sub in progress_bar(brain_subjects):
    meta = brain_data[brain_sub]['meta']
    idxs = brain_data[brain_sub]['sortIdx'][0:num_voxels]
    usedVoxels = np.isin(meta['coordToCol'], idxs)
    cube = np.zeros(usedVoxels.shape)
    cube[usedVoxels] = 1
    dataCube.append(cube)

  dataCube = np.stack(dataCube, axis=3)

  return dataCube 

def get_data_cube(betas, feature_index, brain_data, num_voxels, 
                  brain_subjects=[1,2,3,4,5,6,7,8,9]):
  dataCube = []
  
  for brain_sub in progress_bar(brain_subjects):
    meta = brain_data[brain_sub]['meta']    
    idxs = brain_data[brain_sub]['sortIdx'][0:num_voxels]
    weights = betas[brain_sub-1][0:num_voxels, feature_index]
    usedVoxels = [np.where(meta['coordToCol']==idx) for idx in idxs]
    usedVoxels = tuple(np.concatenate(x) for x in zip(*usedVoxels))
    cube = np.zeros(meta['coordToCol'].shape)
    cube[usedVoxels] = weights
    dataCube.append(cube)

  dataCube = np.stack(dataCube, axis=3)

  return dataCube 

def data_cube_to_color_cube(dataCube, scale_factor=None, binarize=False):
  mask = np.ones(dataCube.shape)  
  if scale_factor is None:
    vals = dataCube[dataCube != 0]
    M = np.median(vals)
    MDM = np.abs(vals-M).mean()
    scale_factor = 1.96*MDM 
  
  # positive weight color range
  rgbPosU=np.array([255, 255, 0])
  rgbPosL=np.array([243, 193, 121])/1.5

  # negative weight color range
  rgbNegL=np.array([192, 211, 255])/1.5
  rgbNegU=np.array([0, 75, 255])

  # initialize the R, G, B color cubes
  colCubeR=np.zeros(dataCube.shape)
  colCubeG=np.zeros(dataCube.shape)
  colCubeB=np.zeros(dataCube.shape)

  # set negative colors
  cube=dataCube.copy()
  negIDX=cube<0;
  posIDX=cube>0;
  cube[posIDX]=0
  cube=np.abs(cube)
  # sf=.50
  # sf = np.abs(dataCube).max()
  # sf = .10
  idx=negIDX

  # rescale the weights by scale_factor
  cubeR=(cube-cube.min()) / scale_factor
  cubeR[cubeR>1]=1
  if binarize: cubeR[cubeR>0]=1;
  newR = (cubeR[idx]*rgbNegU[0]) + ( (1-cubeR[idx])*rgbNegL[0] )
  colCubeR[idx]=colCubeR[idx]+newR*mask[idx]

  cubeG=(cube-cube.min()) / scale_factor
  cubeG[cubeG>1]=1
  if binarize: cubeG[cubeG>0]=1
  newG = (cubeG[idx]*rgbNegU[1]) + ( (1-cubeG[idx])*rgbNegL[1] )
  colCubeG[idx]=colCubeG[idx]+newG*mask[idx]

  cubeB=(cube-cube.min()) / scale_factor
  cubeB[cubeB>1]=1
  if binarize: cubeB[cubeB>0]=1
  newB = (cubeB[idx]*rgbNegU[2]) + ( (1-cubeB[idx])*rgbNegL[2] )
  colCubeB[idx]=colCubeB[idx]+newB*mask[idx]

  # set positive colors
  cube=dataCube.copy()
  cube[negIDX]=0
  idx=posIDX;

  cubeR=(cube-cube.min()) / scale_factor
  cubeR[cubeR>1]=1;
  if binarize: cubeR[cubeR>0]=1;
  newR = (cubeR[idx]*rgbPosU[0]) + (1-cubeR[idx])*rgbPosL[0]
  colCubeR[idx]=colCubeR[idx]+newR*mask[idx]

  cubeG=(cube-cube.min()) / scale_factor
  cubeG[cubeG>1]=1
  if binarize: cubeG[cubeG>0]=1
  newG = (cubeG[idx]*rgbPosU[1]) + (1-cubeG[idx])*rgbPosL[1]
  colCubeG[idx]=colCubeG[idx]+newG*mask[idx]

  cubeB=(cube-cube.min()) / scale_factor
  cubeB[cubeB>1]=1
  if binarize: cubeB[cubeB>0]=1
  newB = (cubeB[idx]*rgbPosU[2]) + (1-cubeB[idx])*rgbPosL[2]
  colCubeB[idx]=colCubeB[idx]+newB*mask[idx]

  return colCubeR, colCubeG, colCubeB

def quick_view_cube(dataCube):
  a, b, c = dataCube.shape
  n = int(np.ceil(np.sqrt(c)))

  fig, axs = plt.subplots(n, n, figsize=(n*4,n*4))
  i = -1
  for row in axs:
    for ax in row:
      i+=1
      if i < c:
        data_slice = dataCube[:,:,i]
        img = np.repeat(data_slice[:,:,np.newaxis],3,axis=2)
        ax.imshow(img, vmin=0, vmax=1)
        ax.axis('off')
      else:
        ax.remove()

def quickViewCubeOverlay(brainCube,dataCube,scale_factor=None, binarize=False,
                         brain_lum=.20, title=''):
  R, G, B = data_cube_to_color_cube(dataCube,
                                    binarize=binarize,
                                    scale_factor=scale_factor)
  a, b, c = dataCube.shape
  n = int(np.ceil(np.sqrt(c)))

  fig, axs = plt.subplots(n, n, figsize=(n*4,n*4))
  i = -1
  for row in axs:
    for ax in row:
      i+=1
      if i < c:
        brain_slice = brainCube[:,:,i]
        brain_slice = brain_slice / brainCube.max() * brain_lum
        img = np.zeros((*brain_slice.shape,3))
        img[:,:,0]=brain_slice*255 + R[:,:,i]*(1-brain_lum)
        img[:,:,1]=brain_slice*255 + G[:,:,i]*(1-brain_lum)
        img[:,:,2]=brain_slice*255 + B[:,:,i]*(1-brain_lum)   

        if img.max() > 0: 
          img = img/255.0
        ax.imshow(img)
        #ax.axis('off')

        if (i==0): 
          ax.set_title('bottom of brain', fontsize=16)
          ax.set_ylabel('back of brain', fontsize=16)

        if (i%n*4)==0:
          ax.set_ylabel('back of brain', fontsize=16)

        if (i==(c-1)): 
          ax.set_title('top of brain', fontsize=16)

        ax.tick_params(
          axis='both',       # changes apply to the x-axis & y-axis
          which='both',      # both major and minor ticks are affected
          bottom=False,      # ticks along the bottom edge are off
          top=False,         # ticks along the top edge are off
          left=False,
          right=False,
          labelbottom=False, # labels along the bottom edge are off
          labelleft=False)   # labels along the left edge are off

      else:
        ax.remove()

  fig.suptitle(title, fontsize=24, y=0.90)

### analysis steps

In [ ]:
import torch
import scipy.io as sio

def step0_prepare_rating_data(teamName, dropRaters=[], dropFeatures=[]):  
  
  download_ratings(teamName, dropRaters=dropRaters, dropFeatures=dropFeatures)

  feature_data = get_feature_data(teamName)

  folder = os.path.join(drive, 'FeatureData')
  if not os.path.exists(folder):
    os.makedirs(folder)

  filename = os.path.join(folder, f'{teamName}_Features.pth.tar')
  torch.save(feature_data, filename)

  filename = os.path.join(folder, f'{teamName}_Features.mat')
  sio.savemat(filename, feature_data)

In [ ]:
def step1_fit_feature_model(teamName, num_voxels=500, 
                            brain_subjects=[1], 
                            testIndividualFeatures=False):
  results = run_full_analysis(teamName, 
                              num_voxels=num_voxels,
                              sub_list=brain_subjects,
                              testIndividualFeatures=testIndividualFeatures)

  pprint(compute_prediction_summary(results['df'],
                                    'brain_prediction',
                                    'encoding_model',
                                    'combo'))
  save_full_results(results)

In [ ]:
def step2_brain_prediction_accuracy(teamName, 
                           num_voxels=500, 
                           method="encoding_model",
                           score="combo"):

  ax = compare_to_mitchell(teamName, 
                           num_voxels=num_voxels, 
                           task="brain_prediction",
                           method=method,
                           score=score,
                           title="Predicting Brain Activity from Features\n")

  return ax;

In [ ]:
def get_results_by_feature(teamName,
                           num_voxels=500,
                           task="brain_prediction",
                           method="encoding_model",
                           score="individual"):
  
  # save file 
  folder = os.path.join(drive,'Results-ByFeature')
  if not os.path.exists(folder): os.makedirs(folder)
  filename = os.path.join(folder, 
                          f'{teamName}_{num_voxels}_results_by_feature.csv')
  
  if os.path.isfile(filename):
    results = pd.read_csv(filename)
  else:
    files = glob(os.path.join(drive, f'./Results-BrainPrediction/*{teamName}*{num_voxels}*features*'))
    assert len(files)==1, f"Oops, expected 1 file, got {len(files)}"
    df = torch.load(files[0])['feat_df']
    features = sorted(df.feat_name.unique())
    subjects = sorted(df.brain_subject.unique())
    results = defaultdict(list)
    for feature in features:
      for subject in subjects:      
        subset = df[(df.feat_name==feature) & (df.brain_subject==subject) & 
                    (df.task==task) & (df.method==method) & (df.scoring==score)]
        assert len(subset)==1770, f"Oops, expected 1770 scores, got {len(subset)}"
        results['feature'].append(feature)
        results['subject'].append(subject)
        results['percent correct'].append(subset.correct.mean())
    results = pd.DataFrame(results)
    results.to_csv(filename, index=False)

  return results  

def step3_determine_best_feature(teamName,
                                 num_voxels=500, 
                                 task="brain_prediction",
                                 method="encoding_model",
                                 score="individual"):
  
  results = get_results_by_feature(teamName,
                                 num_voxels=500, 
                                 task="brain_prediction",
                                 method="encoding_model",
                                 score="individual")
  
  sort_order = results.groupby("feature").mean().sort_values("percent correct", ascending=False).index
  features = sorted(results. feature.unique())

  sns.set(rc={
      'figure.figsize':(14,len(features)*.5),
      'axes.grid': False,
  })
  sns.set(font_scale = 1.5)
  sns.set_style("white")

  ax = sns.barplot(data=results, y="feature", x="percent correct", 
                  order=sort_order, orient = 'h')

  ax.set_xlim([.45,0.85]);
  ax.set_xlabel("percent correct", fontsize=24);
  ax.set_ylabel("feature name", fontsize=24);
  ax.set_title("Accuracy using each individual feature on its own", fontsize=22);
  ax.yaxis.labelpad = 20
  ax.xaxis.labelpad = 20
  ax.title.set_position([.5, 1.02])


In [ ]:
from glob import glob
from functools import lru_cache
from pprint import pprint 

def get_feature_names(teamName, num_voxels=500):
  # get feature-modeling results for this team
  results = get_feature_modeling_results(teamName, num_voxels=num_voxels)
  betas = results['betas']
  featureNames = results['feature_data']['featureNames'].tolist()
  pprint(featureNames)

def get_feature_modeling_results(teamName, num_voxels=500):
  files = glob(os.path.join(drive, 'Results-BrainPrediction', f'*{teamName}*{num_voxels}*'))
  assert len(files)==1, f"Oops, expected 1 file, got {len(files)}"
  filename = files[0]
  print(filename)
  results = torch.load(filename)

  return results 

def step4_visualize_feature_weights(teamName, featureName, num_voxels=500, 
                                    scaling='relative', binarize=False):
  
  # get feature-modeling results for this team
  results = get_feature_modeling_results(teamName, num_voxels=num_voxels)
  betas = results['betas']
  featureNames = results['feature_data']['featureNames'].tolist()
  print(featureNames)
  
  # get gray matter mask for neural data
  brain_data = get_voxel_data()
  brainCube = get_gray_matter_cube(brain_data)
  brain_data.keys(), brainCube.shape

  # get weights for featureName
  feature_index = featureNames.index(featureName)
  dataCube = get_data_cube(betas, feature_index, brain_data, 
                           num_voxels=num_voxels)

  # get scaling factor
  scale_factor = None
  if scaling=='absolute':
    all_betas = np.stack(betas).mean(axis=0)
    vals = np.abs(all_betas[all_betas!=0])
    M = np.median(vals)
    MAD = (vals-M).mean() 
    scale_factor = MAD*3
  
  # finally, plot it
  ax = quickViewCubeOverlay(brainCube.mean(axis=3),dataCube.mean(axis=3),
                            scale_factor=scale_factor, binarize=False,
                            title=f'Feature Weights for {featureName}')
  
  return ax

In [ ]:
def step5_mind_reading_accuracy(teamName, 
                                num_voxels=500, 
                                method="botastic_templates",
                                score="combo"):

  ax = compare_to_mitchell(teamName, 
                           num_voxels=num_voxels, 
                           task="mind_reading",
                           method=method,
                           score=score,
                           title="Mind Reading\n")

  return ax;

In [ ]:
def step6_brain_prediction_same_diff_category(teamName, 
                                              num_voxels=500, 
                                              method="encoding_model",
                                              score="combo"):

  fig,(ax1,ax2) =  plt.subplots(1,2, figsize=(16,8))

  compare_to_mitchell(teamName,
                      num_voxels=num_voxels, 
                      task="brain_prediction",
                      method=method,
                      score=score,
                      accuracy="accuracySameCat",
                      title="Brain-Prediction Same-Category\n",
                      ax=ax1)

  compare_to_mitchell(teamName,
                      num_voxels=num_voxels, 
                      task="brain_prediction",
                      method=method,
                      score=score,
                      accuracy="accuracyDiffCat",
                      title="Brain-Prediction Diff-Category\n",
                      ax=ax2, ylabel="")
  
  plt.show()

  return

In [ ]:
def step7_mind_reading_same_diff_category(teamName, 
                                          num_voxels=500, 
                                          method="botastic_templates",
                                          score="combo"):

  fig,(ax1,ax2) =  plt.subplots(1,2, figsize=(16,8))

  compare_to_mitchell(teamName,
                      num_voxels=num_voxels, 
                      task="mind_reading",
                      method=method,
                      score=score,
                      accuracy="accuracySameCat",
                      title="Mind-Reading Same-Category\n",
                      ax=ax1)

  compare_to_mitchell(teamName,
                      num_voxels=num_voxels, 
                      task="mind_reading",
                      method=method,
                      score=score,
                      accuracy="accuracyDiffCat",
                      title="Mind-Reading Diff-Category\n",
                      ax=ax2, ylabel="")
  
  plt.show()

  return

# run analyses

This notebook shows you the results of two different "brain decoding" analyses (brain-prediction and mind-reading), where your feature ratings were used to analyze the neural responses of Mitchell's fMRI participants.

**Brain-prediction:**

This is where you go toe-to-toe with Mitchell in his original "*predict brain activation from features*" analysis. The goal with brain prediction is to train a feature model to learn what each voxel in the brain responds to. Then, for held out items, whose features are known, predict the brain activation for the held out items.

**Mind-reading:**

We can think of this as "inverting" the brain prediction analysis. Mitchell learned how each voxel was influenced by each feature, and used that to predict neural resposnes based on an items features.

Here, we flip it. Instead we will take the brain response, and use it to predict the features! Then, from predicted features, we'll guess which object was observed (whichever object's true features correlates the most with the predicted features).


# step0_prepare_rating_data

Prepare your rating data for analysis.

In [ ]:
teamName = 'v1report'
dropFeatures = ['color2_Blue',
                'color2_Brown', 'color2_Clear', 'color2_Green ', 'color2_Grey', 
                'color2_Orange', 'color2_Red', 'color2_White', 'color2_Yellow',
                'interaction_nose']
step0_prepare_rating_data(teamName, dropFeatures=dropFeatures)

# step1_fit_feature_model

Fit the feature model (training on all possible leave-2-out pairs for 60 objects = 1770 iterations x 9 fMRI subjects!). This will take a few minutes...

In [ ]:
step1_fit_feature_model(teamName, brain_subjects=[1,2,3,4,5,6,7,8,9])

# step2_brain_prediction_accuracy

Compare your team’s model to Mitchell’s on brain-prediction!  
**Did you win? Put the graph in your lab book with a note about what you think it means (e.g., If you lost, do you think your features were "the wrong features", or that your feature ratings were just too noisy?. If you think your features were the wrong features, what did mitchell get right that you got wrong? Or, if you won, which of your features do you think carried you past Mitchell's feature model? The following steps may help you answer these questions.)**

In [ ]:
step2_brain_prediction_accuracy(teamName);

# step3_determine_best_feature

Determine which of your individual features was the best predictor of brain activation patterns. P**ut the graph in your lab book with a note about what you think it means. (You could also compare to Mitchell’s best feature. His team name is ‘MitchellSemantic’).**

In [ ]:
# skipping this step in this notebook!
# step3_determine_best_feature(teamName)

# step4_visualize_feature_weights

It would be interesting to know which voxels got high weights and which one’s got low weights, and where the high and low weights were in the brain. One way to do that is to visualize the weights in “slices”.  Next answer these questions: Where are the voxels used in our analysis located in the brain? Where are the high weights? Where are the low weights? Does your “best feature” have a different pattern than your “worst feature” (e.g., does it seem more disorganized and random, or does it have hot/cold spots in different places?). **Put some figures in your lab book with comments (just a couple, no need to look at all your features, unless you want to).**

In [ ]:
# get_feature_names(teamName)

In [ ]:
# step4_visualize_feature_weights(teamName, featureName='size')

# step5_mind_reading_accuracy

OK, it’s time to Riverside it (that’s a sports/football term, meaning to change directions on the field). In our case, that means instead of using feature data to predict brain data, we’re going to use brain data to predict feature data. In other words, we want to know whether you can measure someone’s brain activity, use those brain patterns to predict the features of the object, and therefore to determine which object the person was looking at/thinking about. In theory, we can determine what object a person is looking at/thinking about, even if this is the first time we’ve ever measured their brain activity for that item! This is real mind reading, using a “model of how the brain encodes information” to read someone's state of mind (features ~= state of mind).

In this section, we’re going beyond Mitchell’s published work (beyond the cutting edge...that’s fun). Mitchell never did mind reading proper, as we’re about to, so we have to apply our methods to his data for comparison. Conceptually, we’re using the same “leave 2 out” procedure (train on 58 items, test on the remaining 2; repeat 1770 times). But now we’re mapping voxel activations to features values, so we can classify which object was seen from brain activation patterns.

**How well does it work? Put the graph in your lab book with a note about what you think it means.**

In [ ]:
step5_mind_reading_accuracy(teamName);

# step6_brain_prediction_same_diff_category

Whose model does better on brain-prediction with “within-category” vs. "different-category" discriminations, your model or Mitchell’s?


In [ ]:
step6_brain_prediction_same_diff_category(teamName);

# step7_mind_reading_same_diff_category

Whose model does better on mind-reading with “within-category” vs. "different-category" discriminations, your model or Mitchell’s?



In [ ]:
step7_mind_reading_same_diff_category(teamName);

# Wrap it up!

Wrap up your lab book with a few parting thoughts. Where were you successful? What could you do to improve your feature model, and it’s ability to predict brain data, and/or your ability to do mind reading from brain data?